<a href="https://colab.research.google.com/github/NewCodeLearner/HandsOnLLM-Projects/blob/main/08_Advanced_Text_Generation_Techniques_and_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this Chapter, we learn how to code chains using Langchain

In particular, we learn about 3 things:

(1) How to define chains in Langchain?  
(2) How to link a prompt to an LLM using a chain?  
(3) How to chain multiple prompts together

## Install required packages

In [1]:
# %%capture
!pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2 langchain_community
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python==0.2.69

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 MB 13.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.69-cp310-cp310-linux_x86_64.whl size=55266179 sha256=6a7b7f57a53f67adafaae72167888f186d59136544166aa4b9d63b770bb8ed75
  Stored in directory: /root/.cache/pip/wheels/e4/18/46/58b5c613b17c8d000d79ae650980fe871b3b490e04e6faa1c1
Successfully built llama-cpp-python


# Loading an LLM

In [2]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

# If this command does not work for you, you can use the link directly to download the model
# https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2024-12-20 14:47:56--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 13.35.210.114, 13.35.210.77, 13.35.210.61, ...
Connecting to huggingface.co (huggingface.co)|13.35.210.114|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1734965276&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNDk2NTI3Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQxZDg

In [3]:
import os
model_path = '/content/Phi-3-mini-4k-instruct-fp16.gguf'
if not os.path.exists(model_path): raise FileNotFoundError(f"Model file not found at {model_path}")

In [4]:
from langchain import LlamaCpp

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)


In [5]:
llm.invoke("Hello! My name is Girish, what is 2+1?")

''

### Chains

In [6]:
from langchain import PromptTemplate

# Create a prompt template with the "input_prompt" variable
template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

In [7]:
basic_chain = prompt | llm

In [9]:
# Use the chain
basic_chain.invoke(
    {
        "input_prompt": "Hi! My name is Girish. What is 1 + 1?",
    }
)

" Hello Girish! The answer to 1 + 1 is 2. It's a basic arithmetic operation where you add one to another one, resulting in two."

### Multiple Chains

In [10]:
from langchain import LLMChain

# Create a chain for the title of our story
template = """<s><|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

<ipython-input-10-61dd782c6da9>:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")


In [11]:
title.invoke({"summary": "a girl that lost her mother"})

{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of Mother\'s Echo: A Journey Through Loss"'}

In [12]:
# Create a chain for the character description using the summary and title
template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title"]
)
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [13]:
# Create a chain for the story using the summary, title, and character description
template = """<s><|user|>
Create a story about {summary} with the title {title}. The main charachter is: {character}. Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title", "character"]
)
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [14]:
llm_chain = title | character | story

In [15]:
llm_chain.invoke({"summary": "a girl that lost her mother"})

{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of Loss: A Journey Through Grief"',
 'character': ' The central character, Emily, is an introspective and resilient young woman in her late teens who must navigate the complex emotions associated with losing her beloved mother at a tender age. Her journey through grief takes her on a path of self-discovery as she learns to embrace vulnerability while finding strength in cherished memories.',
 'story': ' In "Whispers of Loss: A Journey Through Grief," Emily, an introspective young woman, finds herself grappling with the profound pain of losing her mother at a tender age. As she navigates through the intricate tapestry of emotions that accompany such loss - sorrow, longing, and confusion - Emily embarks on a journey of self-discovery. Through cherished memories and heartfelt moments shared with her mother, she learns to embrace vulnerability as a source of strength, ultimately finding solace in the love they once shared. Ea

### Memory

In [16]:
# Let's give the LLM our name
basic_chain.invoke({"input_prompt": "Hi! My name is Girish. What is 1 + 1?"})

" Hello Girish! The answer to 1 + 1 is 2. It's a basic arithmetic operation where you add one unit to another unit, resulting in two units in total."

In [17]:
# Next, we ask the LLM to reproduce the name
basic_chain.invoke({"input_prompt": "What is my name?"})

" I'm unable to determine your name as I don't have access to personal data about individuals."

### ConversationBuffer

In [18]:
# Create an updated prompt template to include a chat history
template = """<s><|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

In [19]:
from langchain.memory import ConversationBufferMemory

# Define the type of Memory we will use
memory = ConversationBufferMemory(memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-19-9823403f20ac>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [20]:
# Generate a conversation and ask a basic question
llm_chain.invoke({"input_prompt": "Hi! My name is Girish. What is 1 + 1?"})

{'input_prompt': 'Hi! My name is Girish. What is 1 + 1?',
 'chat_history': '',
 'text': " Hello Girish! The answer to what 1 + 1 equals is 2. It's a basic arithmetic operation in mathematics. If you have any more math questions or need assistance with other topics, feel free to ask!"}

In [21]:
# Does the LLM remember the name we gave it?
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Girish. What is 1 + 1?\nAI:  Hello Girish! The answer to what 1 + 1 equals is 2. It's a basic arithmetic operation in mathematics. If you have any more math questions or need assistance with other topics, feel free to ask!",
 'text': " Your name is Girish.\n\nAs for the question about what 1+1 equals, it already has an answer: 2. This basic arithmetic operation demonstrates addition where you combine two ones to get a total of two. If there's anything else you need help with or any further questions, just let me know!"}